data의 text를 통해서 disaster관련 문장이면 target 1 아니면 0

initial method

text -> tokenized 

using pre trained word embedding (Word2Vec)

- GRU (bidirectional)
- 추가로 CNN으로 구성해볼래 (Convolutional Neural Networks for sentence classification)(4.1)

result

input data : text

output data : target(digit 0,1)

model : architecture (GRU) // params : (embedding,GRU의 params)

cost : binary_cross_entropy

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from pandas import DataFrame as df
import nltk
from matplotlib import pyplot as plt

In [2]:
import os
os.getcwd()

'C:\\Users\\IDSL\\Desktop\\파이썬공부\\파이토치\\kaggle challenge\\-Kaggle-Real-or-Not-NLP-with-Disaster-Tweets-day2\\-Kaggle-Real-or-Not-NLP-with-Disaster-Tweets-day2'

In [3]:
data=pd.read_csv('./train.csv',header=0,index_col=0)
data.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
# 1. text data pre processing
# cleanse, tokenize, encoding, vectorize

# 1.1. cleanse
import re
# 이전엔 그냥 str의 replace 활용했는데 #는 못바꾸넹..
# 대문자 -> 소문자
# 특수문자 제거 (구두점, 쉼표 제외)
# http 부분 제거 -> split으로 한 다음에 http 가 있다면 그 부분을 지우고 다시 합치자
def cleanse(sentence):
    result=sentence.lower()
    result=re.sub('[^a-z ,.]','',result)
    result=[i for i in result.split() if 'http' not in i]
    Result=''
    for i in result:
        Result=Result+i+' '
    return Result
data['post_text']=data['text'].apply(lambda i : cleanse(i))
print(data.post_text[:30])

id
1     our deeds are the reason of this earthquake ma...
4               forest fire near la ronge sask. canada 
5     all residents asked to shelter in place are be...
6     , people receive wildfires evacuation orders i...
7     just got sent this photo from ruby alaska as s...
8     rockyfire update california hwy. closed in bot...
10    flood disaster heavy rain causes flash floodin...
13    im on top of the hill and i can see a fire in ...
14    theres an emergency evacuation happening now i...
15    im afraid that the tornado is coming to our ar...
16         three people died from the heat wave so far 
17    haha south tampa is getting flooded hah wait a...
18    raining flooding florida tampabay tampa or day...
19               flood in bago myanmar we arrived bago 
20    damage to school bus on in multi car crash bre...
23                                        whats up man 
24                                       i love fruits 
25                                    summer 

In [5]:
# # of data
len(data) #7613
# train_dev split
from sklearn.model_selection import train_test_split
train_data,val_data=train_test_split(data,test_size=613,shuffle=True) # train data 7000, val data 613

In [6]:
# nltk로 tokenize
# 1.2. tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# train_text_tokenized=word_tokenize(train_data.post_text) #-> error 발생 -> 애초에 word_tokenize는 str을 위한 것

# http가 들어간 sentence들이 있어서 그 부분은 제거해주자.
def word_tokenize_1(sentence):
    j=word_tokenize(sentence)
    result=[i for i in j if 'http' not in i] 
    return result
sentence_tokenized=train_data['post_text'].apply(lambda i : word_tokenize_1(i))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\IDSL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
print(sentence_tokenized.head()) # http가 들어있는 부분들이 있는데 이는 제거하자.-> 제거 完

id
9830     [a, i, started, writing, when, i, couldnt, tal...
5720        [hornybigbadwolf, sets, the, forest, on, fire]
8098     [tennoatax, i, hand, you, a, glass, of, water,...
10813    [ohhmyjoshh, stevenrulles, he, not, gon, na, b...
5207     [.kurtschlichter, yep, considering, that, mill...
Name: post_text, dtype: object


# preprocessing with pytorch

In [44]:
# 1 preprocessing
# torchtext를 활용하면 한방에(tokenize,wordembedding,padding ...)
# SOS,EOS,padding,fix_len : 40 (애초에 twitter는 글자수 제한이 280)
import torchtext
# Field 정의
# Text <- padding 하지 말아라
Text=torchtext.data.Field(sequential=True,use_vocab=True,tokenize=word_tokenize_1,batch_first=True,lower=True,fix_length=None,)
Target=torchtext.data.Field(sequential=False,use_vocab=False,batch_first=True,is_target=True)
Id=torchtext.data.Field(sequential=False,use_vocab=False,batch_first=True)

# Data 정의
# 영어와 ., 만을 남긴 csv 형태로 넘겨주자
# 또한 http부분을 제거
train_data[['post_text','target']].to_csv('train_data.csv',index=True)
train_data_1=torchtext.data.TabularDataset('./train_data.csv',format='csv',fields=[('id',Id),('text',Text),('target',Target)],
                                           skip_header=True)


In [8]:
# vars 객체의 내부 변수가 저장된 딕셔너리 반환
vars(train_data_1[10])

{'id': '19',
 'text': ['flood', 'in', 'bago', 'myanmar', 'we', 'arrived', 'bago'],
 'target': '1'}

In [40]:
model2=Word2Vec(sentence_tokenized,size=300,window=3,min_count=3)


In [43]:
print(len(model2.wv.vocab.keys())) # 4098

4098


In [11]:
from gensim.models import Word2Vec
# https://datascience.stackexchange.com/questions/10695/how-to-initialize-a-new-word2vec-model-with-pre-trained-model-weights
# pretrained model을 활용하는데
# 하고자 하는 수순
# 1. 일단 model을 구축한다(train은 하지 말고, 일단 vocab만 만들어 둔다)
# 2. pretrained model을 가져와서 기존의 model과 intersect를 진행 (기존 model의 vocab만 활용, 그리고 pretrained model의 weight만 활용)
# 3. model 학습

#1
model = Word2Vec(size=300,window=3,min_count=3)
model.build_vocab(sentence_tokenized)
print(len(model.wv.vocab)) # 4098

#2
model.intersect_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary=True)
print(len(model.wv.vocab))

print(model.wv.most_similar('disaster'))
print(model.wv.vectors.shape)

#3
# If sentences is the same corpus that was provided to build_vocab() earlier, 
# you can simply use total_examples=self.corpus_count.
model.train(sentence_tokenized,total_examples=model.corpus_count,epochs=500)
print(model.wv.most_similar('disaster'))
print(model.wv.vectors.shape)

4098
4098
[('disasters', 0.7751655578613281), ('calamity', 0.740983784198761), ('catastrophe', 0.7316239476203918), ('tragedy', 0.5871202945709229), ('devastation', 0.5772956013679504), ('earthquake', 0.5394471883773804), ('tsunami', 0.5323168635368347), ('floods', 0.5304960012435913), ('catastrophic', 0.5219818353652954), ('flood', 0.5129648447036743)]
(4098, 300)


# gensim.models.Word2Vec.intersect_word2vec_format
Merge the input-hidden weight matrix from the original C word2vec-tool format given, where it intersects with the current vocabulary. (No words are added to the existing vocabulary, but intersecting words adopt the file’s weights, and non-intersecting words are left alone.)
binary is a boolean indicating whether the data is in binary word2vec format.
-> google 같은 경우엔 binary true로 해야겠지.
lockf is a lock-factor value to be set for any imported word-vectors; the default value of 0.0 prevents further updating of the vector during subsequent training. Use 1.0 to allow further training updates of merged vectors.

cf)
https://medium.com/@omicro03/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC-nlp-14%EC%9D%BC%EC%B0%A8-word2vec-%EC%8B%A4%EC%8A%B52-8e518a358b6c

- model = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)

https://m.blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221349385092&proxyReferer=https%3A%2F%2Fwww.google.com%2F

In [82]:
# torchtext로 구성해낸다.
# 관건 align 작업
model.wv.save_word2vec_format('C:/Users/IDSL/Desktop/파이썬공부/파이토치/kaggle challenge/-Kaggle-Real-or-Not-NLP-with-Disaster-Tweets-day2/-Kaggle-Real-or-Not-NLP-with-Disaster-Tweets-day2/wv')
vectors=torchtext.vocab.Vectors('wv',cache='C:/Users/IDSL/Desktop/파이썬공부/파이토치/kaggle challenge/-Kaggle-Real-or-Not-NLP-with-Disaster-Tweets-day2/-Kaggle-Real-or-Not-NLP-with-Disaster-Tweets-day2/') #'./wv'모델을 읽어온다.
# 안될 때엔 변수 싹 다 지우고 다시 해라...
# 이 짓 너무 했다..

In [100]:
model.wv.index2word==list(vectors.stoi.keys())

True

In [204]:
# https://discuss.pytorch.org/t/aligning-torchtext-vocab-index-to-loaded-embedding-pre-trained-weights/20878/2
 # load embeddings using torchtext
Text.build_vocab(train_data_1,min_freq=3,vectors=vectors)
# Text.vocab.set_vectors(vectors.stoi,vectors.vectors,vectors.dim)

In [183]:
# 다 같은 것을 입증.
# freq가 달라도, vectors에 있던 것만 들어가게 된다.
result=[]
for vocab,_ in vectors.stoi.items():
    r=vectors.vectors[_]==Text.vocab.vectors[Text.vocab.stoi[vocab]]
    result.append(np.all(r.numpy()))
print(np.all(np.array(result)))

True


In [210]:
batch_size=2
train_loader=torchtext.data.Iterator(train_data_1,batch_size=batch_size)
batch=next(iter(train_loader))

In [211]:
print(batch.text)
print(batch.target)

tensor([[   9,  213,    0,  669,    6,  148,   28, 2415,   28,  560,    5,  889,
            7,   15,  814,    8,    0,  527,    0, 2809,    0,   44],
        [ 414,   24,   19,    0,  531,  481,   10,    0,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1]])
tensor([0, 1])


In [212]:
eb=nn.Embedding(4100,300).from_pretrained(Text.vocab.vectors)
f=nn.Sequential(nn.Conv1d(300,100,2,1,1),nn.ReLU())

In [228]:
e=eb(batch.text).transpose(1,2)
z=f(e)
z.max(dim=2)[0].shape


torch.Size([2, 100])

In [206]:
class cnn_sentiment(nn.Module):
    def __init__(self,embedding_dim,num_of_filter):
        super().__init__()
        self.embedd=nn.Embedding(Text.vocab.vectors.shape[0],embedding_dim).from_pretrained(Text.vocab.vectors)
        self.conv1=nn.Sequential(nn.Conv1d(embedding_dim,100,2,1,1),nn.ReLU())
        self.conv2=nn.Sequential(nn.Conv1d(embedding_dim,100,3,1,1),nn.ReLU())
        self.conv3=nn.Sequential(nn.Conv1d(embedding_dim,100,4,1,1),nn.ReLU())
        self.fc=nn.Sequential(nn.dropout(0.5),nn.Linear(300,2),nn.ReLU())